# Выбор модели. Валидация. Настройка гиперпараметров.

Уже научились читать и чистить данные. Познакомились с некоторыми моделями машинного обучения, а также метриками для задач классификации и регрессии.

**Но как понять, что мы не переобучились? Достаточной ли обобщающей способностью обладает модель? На сколько хорош выбранный _пайплайн_ в целом?**

<div style="text-align:center">
<img src="src/overfitting.png" width="650px">
</div>

Проблема выбора:
- модели алгоритмов
- гиперпараметров
- способа предобработки данных


**Валидация** - процедура эмпирического оценивания обобщающей способности модели, которая используется для сравнения между собой различных моделей и выбора наилучшей для конкретной задачи.

---------------------------

## [Разбиение данных](https://education.yandex.ru/handbook/ml/article/kross-validaciya)

### Общая схема разбиения данных: Train / Validation / Test Set

- **Обучающая выборка** - Training set (70%)
  - обучение модели (настойка ее параметров / весов)
- **Валидационная выборка** - Validation set (15%)
  - выбор пайплайна (модели / гиперпараметров / способа обработки данных)
  - иногда: локальный контроль
- **Тестовая выборка** - Test set (15%)
  - оценка качества алгоритма
  - иногда: итоговая оценка алгоритма
  - _лучше выбрать в самом начале и вспомнить только на финальном этапе_

<div style="text-align:center">
<img src="src/data_split.png" width="650px">
</div>

**Оптимизация качества моделей и их сравнение на одном и том же множестве, может неявно заложить в модели информацию о тестовом множестве, что может привести к получению результатов хуже ожидаемых на новых данных.**

Реализация разбиения в [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

```

sklearn.model_selection.train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)

```
- ``train_size/test_size: Optional[Union[int, float]]`` - $100: 70/30 \rightarrow 30: 50/50$ - для начального приближения; здравый смысл далее;
  - большее обучение:
    - более репрезентативная обучающая выборка
  - больший контроль:
    - более надежная оценка качества
- ``random_state`` - воспроизводимость результатов
- ``shuffle: bool`` - перемешивание сэмплов.
  - Важно смотреть на природу данных!
- ``stratify`` - _(стратификация)_ разбиение на трейн и тест, сохраняющее соотношение классов, представленное в исходном датасете.
  - Важно с случае сильно несбалансированных данных!

<div style="text-align:center">
<img src="src/general-population.svg" width="650px">
</div>

Для обучения и выбора модели есть много вариантов разбиения:

------------------

### [Кросс-Валидация](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

Зачастую, когда говорят о кросс-валидации имеют в виду именно  метод [K-Fold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).

Он устроен следующим образом:

- Фиксируем целое число K (обычно от 5 до 10 и нечетное)
- Делим выборку на K примерно равных частей (фолдов)
- Цикл по i = 1...K:
  - Использовать i-ую часть для валидации, объединение остальных - для обучения.
- Финальный скор модели получается либо усреднением K получившихся тестовых результатов, либо измеряется на отложенном тестовом множестве, не участвовавшем в кросс-валидации.

<div style="text-align:center">
<img src="src/CV.png" width="650px">
</div>

В scikit-learn реализовано несколько дополнительных методов:
  - [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) - в отличие от K-Fold принимает на вход не только данные, но и модель, и прочие параметры.
    - на выходе оценивает ее качество, а не возвращает разбиение данных.
  - [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) - это метод K-Fold, использующий стратификацию при разбиении на фолды: каждый фолд содержит примерно такое же соотношение классов, как и всё исходное множество.
    - Такой подход может потребоваться в случае, например, очень несбалансированного соотношения классов, когда при обычном random split некоторые фолды могут либо вообще не содержать семплов каких-то классов, либо содержать их слишком мало.
   
### [Leave-One-Out / LOO](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html)

- Вырождение K-Fold при K соответсвующем размеру выборки (каждый фолд состоит ровно из одного сэмлпа).

<div style="text-align:center">
<img src="src/LOOCV.png" width="650px">
</div>

- Большие k:
  + (+) надёжнее оценка качества
  + (+) обучающая выборка больше походит на все данные
  - (-) вычислительно затратно
    - обычно применяют
      -  когда данных достаточно мало
      -  при наличии большого количества вычислительных ресурсов, позволяющих проводить все K итераций параллельно.
  - (-) не любое качество адекватно оценивается на маленьких подвыборках

### [Бутстреп (Bootstrap)](https://v-marco.github.io/psmo_book/content/bootstrap/seminar.html)

-  это псевдовыборка с возвращением, формируется как подвыборка полного объёма, на которой производится обучение модели, а на остальных объектах (которые не попали в обучение) – контроль.

<div style="text-align:center">
<img src="src/bootstrap.png" width="650px">
</div>


В контроль попадает:

$$ \left ( 1 - \frac{1}{m} \right )^m \approx e^{-1} \approx 0.37$$

- Модель учится на выборке того же объема, что и итоговая
- Похожа на исходную с точки зрения распределенения
- Имеются дубликаты

### Контроль по времени. [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html)

Контроль качества в задаче прогнозирования временных рядов имеет существенную особенность - данные не должны пересекаться по времени: тренировочные данные должны идти до валидационных, а валидационные — до тестовых. С учётом этих особенностей фолды в кросс-валидации для временных рядов располагаются вдоль временной оси так, как показано на следующей картинке:

<div style="text-align:center">
<img src="src/time_series.png" width="650px">
</div>

#### FYI: [Больше разбиений](https://scikit-learn.org/stable/api/sklearn.model_selection.html)


#### На что обратить внимание?

- высокое качество на обучении и низкое на контроле - overfitting
  - сложность модели, шум, нерепрезентативность
- низкое качество на обучении и контроле -  underfitting
- нужно ли перемешивать данные? перемешали ли вы их?
- не подглядываете в тест в процессе обработки/обучения/подбора гиперпараметров?
- временные ряды


------------------

## Гиперпараметры

Чем гиперпараметры отличаются от параметров модели?
  - Параметры модели настраиваются в процессе обучения модели на данных
  - Гиперпараметры - характеристики модели, которые фиксируются до начала обучения модели (экспертно или переборно)

Простейшие примеры гиперпараметров:
 - параметр регуляризации
 - темп обучения градиентного спуска (learning rate)


### Простейшие стратегии подбора гиперпараметров. Переборные методы.

#### Поиск по сетке. [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
Самым наивным способом поиска оптимального набора гиперпараметров является поиск по сетке. Метод выполняет полный перебор по изначально заданному подмножеству гиперпараметров выбранного алгоритма. Так как пространство поиска гиперпараметров выбранной модели машинного обучения может быть непрерывным и неограниченным, ML-разработчик сам должен установить границы и провести дискретизацию значений. Качество алгоритма поиска обычно определяется с помощью перекрестной проверки или отложенного набора данных. В результате своей работы алгоритм **поиска по сетке** выдаёт модель с гиперпараметрами, на которых был достигнут наилучший результат в процессе проверки.

Основным недостатком данного метода является *проклятие размерности* -- проблема экспоненциального роста необходимых экспериментальных данных в зависимости от роста размерности пространства. Однако обычно метод довольно легко поддается распараллеливанию, так как зачастую гиперпараметры не зависят друг от друга.  


#### Случайный поиск. [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
Основной идей случайного поиска является независимое сэмлирование из заранее определенного подмножества гиперпараметров до тех пор, пока не будет исчерпан заранее определенный бюджет, например, максимальное количество итераций. В отличие от предыдущего метода от разработчика не требуется проводить дискретизацию непрерывных значений, нужно лишь ограничить пространство поиска. Так же, как и  *поиск по сетке*, *случайный поиск* поддерживает параллельную работу, так как вычисления могут выполняться независимо, а также позволяет использовать априорные знания об оптимизируемых параметрах путем указания распределения на них. *Случайный поиск* превосходит *поиск по сетке* в случае, если на качество модели оказывает влияние лишь малое количество параметров.

<div style="text-align:center">
<img src="src/grid_vs_random_search.png" width="650px">,
</div>

График функция $g(x)$ изображен зеленым цветом сверху квадрата, а график функции $h(y)$ представлен на левой боковой части квадрата. В случае использования поиска по сетке, из $9$ измерений только $3$  протестируют функцию $g(x)$ в разных точках. При использовании случайного поиска все $9$ измерений эффективно исследуют $g(x)$ в $9$ разных точках. Такое поведение метода *поиска по сетке* в сравнении с методом *случайного поиска* является скорее правилом, чем исключением, в многомерной гиперпараметрической оптимизации.

[**Примеры использования.**](https://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-optimization)

**Другие библиотеки**: [Optuna](https://optuna.org/), [scikit-optimize](https://scikit-optimize.github.io/stable/index.html), [hyperopt](http://hyperopt.github.io/hyperopt/).


### Байесовская оптимизация

Поиск по сетке и случайный поиск не используют информацию, полученную на основе предыдущих наблюдений. Байесовская оптимизация призвана устранить этот существенный недостаток. Концептуально имеется две области для выбора новых точек: рассмотрение точек в малоисследованных на текущий момент областях (_exploration_ стратегия) и рассмотрение точек в достаточно изученных областях, где с высокой вероятностью может находится оптимум (_exploitation_ стратегия). Для этого модель байесовской оптимизации использует 2 основных компонента:

  - ***Вероятностная модель*** исследуемой функции. Она приближает распределение значений целевой функции на основе имеющихся данных (часто используют гауссовские процессы).
  - ***Acquisition функция***, которая на основе некоторой статистической информации, полученной из вероятностной модели функции, определяет, в какой точке вычислять значение целевой функции на следующем шаге. Она балансирует между обозначенными стратегиями:
    - исследует точки с большой дисперсией вероятностной модели функции
    - исследует точки с большим средним вероятностной модели функции

Один из примеров Acquisition функции - **Upper confidence bound (UCB)**:
$$UCB[\mathbf{x}] = \mu(\mathbf{x}) + \beta^{1/2} \sigma(\mathbf{x}),$$
где $\mathbf{x}$ - вектор значений гиперпараметров, $\mu(\cdot)$ и $\sigma(\cdot)$ - среднее и стандартное отклонение вероятностой модели, а $\beta$ - положительный параметр, который позволяет балансировать между _exploration_ и _exploitation_ стратегиями.

<div style="text-align:center">
<img src="src/bayes.png" width="650px">,
</div>

Основной проблемой байесовской оптимизации является отсутствие возможности распараллеливания процесса, так как вычисление каждой новой точки использует информацию о точках, полученных на предыдущих итерациях.

[Successive Halving](https://arxiv.org/pdf/1502.07943), [HyperBand](https://arxiv.org/pdf/1603.06560), [BOHB](https://arxiv.org/pdf/1807.01774) и [обзор](https://habr.com/ru/companies/skillfactory/articles/528240/).

### Эвристические алгоритмы.

Эвристика - алгоритм решения задачи, включающий практический метод, не являющийся гарантированно точным или оптимальным, но достаточный для решения поставленной задачи.

- Эвристический алгоритм, в отличие от точного, обладает следующими характерными особенностями:
  - При использовании алгоритма результат не всегда будет гарантированно точным;
  - В некоторых случаях алгоритм может привести к неверному результату;
  - Возможен «пропуск цели», то есть решение не будет найдено, даже если известно, что оно заведомо существует;
  - В ряде случаев может быть даже доказано, что эвристический алгоритм формально неверен. Но, несмотря на это, приемлем, если он дает неверный результат только в отдельных случаях, или же дает не абсолютно точный, но все же приемлемый результат.


Примеры: 
- [Генетический алгоритм](https://ru.wikipedia.org/wiki/%D0%93%D0%B5%D0%BD%D0%B5%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC)
- [Эволюционная стратегия](https://ru.wikipedia.org/wiki/%D0%AD%D0%B2%D0%BE%D0%BB%D1%8E%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%B5%D0%B3%D0%B8%D1%8F)
- [Дифференциальная эволюция](https://ru.wikipedia.org/wiki/%D0%94%D0%B8%D1%84%D1%84%D0%B5%D1%80%D0%B5%D0%BD%D1%86%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%8D%D0%B2%D0%BE%D0%BB%D1%8E%D1%86%D0%B8%D1%8F)

### Ссылки.
- [Кросс-валидация handbook](https://education.yandex.ru/handbook/ml/article/kross-validaciya)
- [Кросс-валидация nerc](https://neerc.ifmo.ru/wiki/index.php?title=%D0%9A%D1%80%D0%BE%D1%81%D1%81-%D0%B2%D0%B0%D0%BB%D0%B8%D0%B4%D0%B0%D1%86%D0%B8%D1%8F)
- [Лекция: Контроль качества и выбор модели. Дьяконов А.Г.](https://github.com/Dyakonov/MSUML/blob/main/2021autumn/ML040_control_202110a_______.pdf)
- [Видео лекции: Контроль качества и выбор модели. Дьяконов А.Г.](https://www.youtube.com/watch?v=Q2jJ8_oU3-s&list=PLhe7c-LCgl4Ic-FRawaaEhUmDCQmGMtzx&index=7)
- [Cross-Validation in Machine Learning: How to Do It Right](https://neptune.ai/blog/cross-validation-in-machine-learning-how-to-do-it-right)
- [Подбор гиперпараметров](https://education.yandex.ru/handbook/ml/article/podbor-giperparametrov)
- [Optuna: подбор гиперпараметров для вашей модели](https://practicum.yandex.ru/blog/optuna-podbor-giperparametrov/)
- [HyperBand и BOHB. Понимание современных алгоритмов оптимизации гиперпараметров](https://habr.com/ru/companies/skillfactory/articles/528240/)
- [Hyperparameter optimization for Neural Networks](http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html)